In [ ]:
from argparse import Namespace
import torch
import numpy as np
from torchid.dynonet.module.lti import SisoLinearDynamicalOperator
from torchid.dynonet.module.static import SisoStaticNonLinearity
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import nonlinear_benchmarks

In [ ]:
cfg = {
    "n_b": 8,
    "n_a": 8,
    "lr": 2e-4,
}
cfg = Namespace(**cfg)

In [ ]:
# Load the benchmark data
train_val, test = nonlinear_benchmarks.WienerHammerBenchMark()
sampling_time = train_val.sampling_time
u_train, y_train = train_val
u_train = u_train.reshape(-1, 1) # T, 1
y_train = y_train.reshape(-1, 1) # T, 1

In [ ]:
# Rescale data (Z-score)
scaler_u = StandardScaler()
u = scaler_u.fit_transform(u_train)

scaler_y = StandardScaler()
y = scaler_y.fit_transform(y_train)

In [ ]:
model = torch.nn.Sequential(
    SisoLinearDynamicalOperator(cfg.n_b, cfg.n_a, n_k=1),
    SisoStaticNonLinearity(n_hidden=10, activation="tanh"),
    SisoLinearDynamicalOperator(cfg.n_b, cfg.n_a),
)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr)

In [ ]:
# Configure optimizer
opt = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
# Load data
u = torch.tensor(u).unsqueeze(0).float() # B=1, T, 1
y = torch.tensor(y).unsqueeze(0).float() # B=1, T, 1

In [ ]:
LOSS = []
# Train loop
for itr in tqdm(range(5000)):

    y_pred = model(u)
    loss = torch.nn.functional.mse_loss(y, y_pred)

    loss.backward()
    opt.step()

    opt.zero_grad()
    if itr % 100 == 0:
        print(loss.item())
    LOSS.append(loss.item())

In [ ]:
checkpoint = {
    'scaler_u': scaler_u,
    'scaler_y': scaler_y,
    'model': model.state_dict(),
    'LOSS': np.array(LOSS),
    'cfg': cfg
}

torch.save(checkpoint, "ckpt.pt")